In [ ]:
import sys
sys.path.append('../')
import os

In [ ]:
input_dir  = os.path.expanduser('../output/openstack/')
output_dir = '../output/openstack/'  

In [ ]:
log_sequence_file = output_dir + "openstack_sequence.csv"

In [ ]:
def generate_train_test(openstack_sequence_file, n=None, ratio=0.3):
    instanceId_label_dict = {}
    instanceId_label_file = os.path.join(input_dir, "unique_label_filtered.csv")
    instanceId_df = pd.read_csv(instanceId_label_file)
    for _, row in tqdm(instanceId_df.iterrows()):
        instanceId_label_dict[row["instance_id"]] = 1 if row["Labels"] == "ANOMALY" else 0

    seq = pd.read_csv(openstack_sequence_file)
    seq["Labels"] = seq["instance_id"].apply(lambda x: instanceId_label_dict.get(x))  # add label to the sequence of each blockid

    normal_seq = seq[seq["Labels"] == 0]["EventSequence"]
    normal_seq = normal_seq.sample(frac=1, random_state=20)  # shuffle normal data

    abnormal_seq = seq[seq["Labels"] == 1]["EventSequence"]
    normal_len, abnormal_len = len(normal_seq), len(abnormal_seq)
    train_len = n if n else int(normal_len * ratio)
    print("normal size {0}, abnormal size {1}, training size {2}".format(normal_len, abnormal_len, train_len))

    train = normal_seq.iloc[:train_len]
    test_normal = normal_seq.iloc[train_len:]
    test_abnormal = abnormal_seq

    df_to_file(train, output_dir + "train")
    df_to_file(test_normal, output_dir + "test_normal")
    df_to_file(test_abnormal, output_dir + "test_abnormal")
    print("generate train test data done")


def df_to_file(df, file_name):
    with open(file_name, 'w') as f:
        for row in df:
            f.write(' '.join([str(ele) for ele in eval(row)]))
            f.write('\n')


In [ ]:
if __name__ == "__main__":
    
    generate_train_test(log_sequence_file, n=2050)